In [3]:
import os
import time
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import requests

# ------------------------------
# Параметры Telegram
# ------------------------------
mentor_chat_id = -4896541130
doctor_chat_id = -4959114772

TELEGRAM_BOT_TOKEN = os.environ.get("TELEGRAM_BOT_TOKEN", "8278992300:AAG7jZDoxJiIV4xKSZ9tZb8C2sKgCr9gEVQ")
MENTOR_TELEGRAM_CHAT_ID = os.environ.get("MENTOR_TELEGRAM_CHAT_ID", str(mentor_chat_id))
DOCTOR_TELEGRAM_CHAT_ID = os.environ.get("DOCTOR_TELEGRAM_CHAT_ID", str(doctor_chat_id))
NOTIFY_COOLDOWN_MINUTES = 30  # минимальный интервал между alert одного уровня

In [4]:
# ------------------------------
# Правило зоны
# ------------------------------
def assign_zone(hrv_mean, hr_mean, activity_mean, waso_min, sleep_eff):
    if (hrv_mean < 35) or (hr_mean > 95) or (waso_min > 60) or (sleep_eff < 0.70):
        return 2  # красная
    if (hrv_mean < 45) or (hr_mean > 80) or (waso_min > 30) or (sleep_eff < 0.80):
        return 1  # жёлтая
    return 0  # зелёная

In [5]:
# ------------------------------
# Генератор данных для нескольких дней
# ------------------------------
def generate_timeseries_days(start_date=None, days=7, step_minutes=1,
                             baseline_hrv=55, baseline_hr=68, baseline_activity=0.6,
                             seed=42):
    np.random.seed(seed)
    if start_date is None:
        start_date = datetime.utcnow()

    rows = []
    for day in range(days):
        stress_factor = day / days
        # дневные метрики сна
        waso = np.random.normal(20 + 40*stress_factor, 5)
        tst = np.random.normal(420 - 30*stress_factor, 10)
        sleep_eff = np.clip(tst / (tst + waso), 0.5, 0.98)

        day_start = start_date + timedelta(days=day)
        for minute in range(0, 24*60, step_minutes):
            timestamp = day_start + timedelta(minutes=minute)
            hrv_mean = baseline_hrv - np.random.normal(0,2) - stress_factor*15*np.random.rand()
            hr_mean = baseline_hr + np.random.normal(0,2) + stress_factor*15*np.random.rand()
            activity_mean = baseline_activity + np.random.normal(0,0.05) - stress_factor*0.2*np.random.rand()

            zone = assign_zone(hrv_mean, hr_mean, activity_mean, waso, sleep_eff)
            rows.append({
                'timestamp': timestamp.isoformat(),
                'day': day+1,
                'minute': minute,
                'hrv_mean': round(hrv_mean,2),
                'hr_mean': round(hr_mean,2),
                'activity_mean': round(activity_mean,3),
                'waso_min': round(waso,2),
                'tst_min': round(tst,1),
                'sleep_efficiency': round(sleep_eff,3),
                'zone': zone
            })

    df = pd.DataFrame(rows)
    return df

In [6]:
# ------------------------------
# Функция отправки Telegram сообщения
# ------------------------------
def send_telegram_to_mentor(message):
    if not TELEGRAM_BOT_TOKEN or not MENTOR_TELEGRAM_CHAT_ID:
        print("[ALERT not sent] " + message)
        return False
    url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
    resp = requests.post(url, json={"chat_id": MENTOR_TELEGRAM_CHAT_ID, "text": message})
    return resp.ok

def send_telegram_to_doctor(message):
    if not TELEGRAM_BOT_TOKEN or not DOCTOR_TELEGRAM_CHAT_ID:
        print("[ALERT not sent] " + message)
        return False
    url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
    resp = requests.post(url, json={"chat_id": DOCTOR_TELEGRAM_CHAT_ID, "text": message})
    return resp.ok

In [7]:
# ------------------------------
# Мониторинг новых строк и уведомления
# ------------------------------
def monitor_dataframe(df):
    last_notify_time = {1: None, 2: None}
    last_zone = None

    for _, row in df.iterrows():
        zone = int(row['zone'])
        ts = row['timestamp']
        hrv = row['hrv_mean']
        hr = row['hr_mean']
        activity = row['activity_mean']
        waso = row['waso_min']
        se = row['sleep_efficiency']

        now = datetime.utcnow()

        if zone == 1:
            print("Зона 1")
            print(f"⚠️ Жёлтая зона {ts}\nHRV={hrv}, HR={hr}, Activity={activity}, "
                       f"WASO={waso}, SleepEff={se}\nДействие: уведомить Наставника.")
            print()
            allow = last_zone != 1 or last_notify_time[1] is None or \
                    (now - last_notify_time[1] >= timedelta(minutes=NOTIFY_COOLDOWN_MINUTES))
            if allow:
                msg = (f"⚠️ Жёлтая зона {ts}\nHRV={hrv}, HR={hr}, Activity={activity}, "
                       f"WASO={waso}, SleepEff={se}\nДействие: уведомить Наставника.")
                send_telegram_to_mentor(msg)
                last_notify_time[1] = now
        elif zone == 2:
            print("Зона 2")
            print(f"⛑️ КРАСНАЯ ЗОНА {ts}\nHRV={hrv}, HR={hr}, Activity={activity}, "
                       f"WASO={waso}, SleepEff={se}\nДействие: немедленно уведомить врача/службы.")
            print()
            allow = last_zone != 2 or last_notify_time[2] is None or \
                    (now - last_notify_time[2] >= timedelta(minutes=NOTIFY_COOLDOWN_MINUTES))
            if allow:
                msg = (f"⛑️ КРАСНАЯ ЗОНА {ts}\nHRV={hrv}, HR={hr}, Activity={activity}, "
                       f"WASO={waso}, SleepEff={se}\nДействие: немедленно уведомить врача/службы.")
                send_telegram_to_doctor(msg)
                last_notify_time[2] = now

        last_zone = zone
        time.sleep(0.01)  # имитация «потока» (10мс на строку)

In [8]:
# ------------------------------
# Запуск генерации + мониторинга
# ------------------------------
if __name__ == "__main__":
    days = 7
    print(f"Генерируем данные для {days} дней...")
    df = generate_timeseries_days(days=days)
    csv_file = "person_timeseries_multiday.csv"
    df.to_csv(csv_file, index=False)
    print(f"✅ CSV сохранён: {csv_file}")

    print("Запускаем мониторинг и уведомления...")
    monitor_dataframe(df)

Генерируем данные для 7 дней...


/tmp/ipython-input-3804632608.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start_date = datetime.utcnow()


✅ CSV сохранён: person_timeseries_multiday.csv
Запускаем мониторинг и уведомления...


/tmp/ipython-input-2441445647.py:17: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow()


Выходные данные были обрезаны до нескольких последних строк (5000).
Зона 2
⛑️ КРАСНАЯ ЗОНА 2025-09-29T00:25:02.834838
HRV=37.59, HR=77.2, Activity=0.646, WASO=64.78, SleepEff=0.857
Действие: немедленно уведомить врача/службы.

Зона 2
⛑️ КРАСНАЯ ЗОНА 2025-09-29T00:26:02.834838
HRV=44.84, HR=81.47, Activity=0.393, WASO=64.78, SleepEff=0.857
Действие: немедленно уведомить врача/службы.

Зона 2
⛑️ КРАСНАЯ ЗОНА 2025-09-29T00:27:02.834838
HRV=50.04, HR=71.95, Activity=0.642, WASO=64.78, SleepEff=0.857
Действие: немедленно уведомить врача/службы.

Зона 2
⛑️ КРАСНАЯ ЗОНА 2025-09-29T00:28:02.834838
HRV=54.32, HR=72.59, Activity=0.653, WASO=64.78, SleepEff=0.857
Действие: немедленно уведомить врача/службы.

Зона 2
⛑️ КРАСНАЯ ЗОНА 2025-09-29T00:29:02.834838
HRV=54.63, HR=69.35, Activity=0.469, WASO=64.78, SleepEff=0.857
Действие: немедленно уведомить врача/службы.

Зона 2
⛑️ КРАСНАЯ ЗОНА 2025-09-29T00:30:02.834838
HRV=52.89, HR=76.91, Activity=0.53, WASO=64.78, SleepEff=0.857
Действие: немедленно